In [37]:
import os, json, time, sys
from __future__ import annotations
from typing import List
from dataclasses import dataclass, field
from gridappsd import GridAPPSD, topics as t
from gridappsd_cim import *
import gridappsd_cim as cim
#from gridappsd_cim.loaders import DistributedModel
from gridappsd_cim.loaders.blazegraph_connection import BlazeGraphConnection

ModuleNotFoundError: No module named 'gridappsd_cim.loaders.blazegraph_connection'

In [2]:
os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-cim-profile'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'
gapps = GridAPPSD()
assert gapps.connected

log = gapps.get_logger()


In [3]:

#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
#feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [4]:


    
#feeder = Model(f)
#feeder.add_to_catalog(t.mRID, t)

In [5]:
@dataclass
class SwitchArea:
    area_id: str
    addressable_equipment: dict[str, object] = field(default_factory=dict)
    unaddressable_equipment: dict[str, object] = field(default_factory=dict)
    boundary_switches: dict[str, object] = field(default_factory=dict)
    connectivity_nodes: dict[str, object] = field(default_factory=dict)
    secondary_areas: list[object] = field(default_factory=list)
    typed_catalog: dict[type, dict[str,object]] = field(default_factory=dict)
    
    # Initialize empty CIM objects for all equipment in switch area
    def initialize_switch_area(self, switch_msg: dict):
        feeder_id = self.area_id.split('.')[0] # get feeder mRID from area id
        
        addr_equip = initialize_objects(feeder_id, switch_msg['addressable_equipment'])
        for obj in addr_equip: 
            DistributedModel.add_to_catalog(obj, self.addressable_equipment)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        unaddr_equip = initialize_objects(feeder_id, switch_msg['unaddressable_equipment'])
        for obj in unaddr_equip: 
            DistributedModel.add_to_catalog(obj, self.unaddressable_equipment)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        conn_nodes = initialize_objects(feeder_id, switch_msg['connectivity_node'])
        for obj in conn_nodes: 
            DistributedModel.add_to_catalog(obj, self.connectivity_nodes)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        bound_sw = initialize_objects(feeder_id, switch_msg['boundary_switches'])
        for obj in bound_sw: 
            DistributedModel.add_to_catalog(obj, self.boundary_switches)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        
        sa_index = -1
        for sec_area_msg in switch_msg['secondary_areas']:
           # Add switch area
            sec_area = SecondaryArea(feeder_mrid + '.' + str(sa_index))
            sec_area.initialize_secondary_area(sec_area_msg)
            self.secondary_areas.append(sec_area)
            # Add switch area unaddressable equipment to feeder unaddressable equipment
            self.unaddressable_equipment.update(sec_area.unaddressable_equipment)
            sa_index = sa_index + 1

In [6]:
@dataclass
class SecondaryArea:
    area_id: str
    distribution_transformer: dict[str, object] = field(default_factory=dict)
    addressable_equipment: dict[str, object] = field(default_factory=dict)
    unaddressable_equipment: dict[str, object] = field(default_factory=dict)
    connectivity_nodes: dict[str, object] = field(default_factory=dict)
    secondary_areas: list[object] = field(default_factory=list)
    typed_catalog: dict[type, dict[str,object]] = field(default_factory=dict)
    
    # Initialize empty CIM objects for all equipment in secondary area
    def initialize_secondary_area(self, switch_msg: dict):
        feeder_id = self.area_id.split('.')[0] # get feeder mRID from area id
        addr_equip = initialize_objects(feeder_id, switch_msg['addressable_equipment'])
        for obj in addr_equip: 
            DistributedModel.add_to_catalog(obj, self.addressable_equipment)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        unaddr_equip = initialize_objects(feeder_id, switch_msg['unaddressable_equipment'])
        for obj in unaddr_equip: 
            DistributedModel.add_to_catalog(obj, self.unaddressable_equipment)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        conn_nodes = initialize_objects(feeder_id, switch_msg['connectivity_node'])
        for obj in conn_nodes: 
            DistributedModel.add_to_catalog(obj, self.connectivity_nodes)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)
        xfmr = initialize_objects(feeder_id, switch_msg['distribution_transformer'])
        for obj in xfmr: 
            DistributedModel.add_to_catalog(obj, self.distribution_transformer)
            DistributedModel.add_to_typed_catalog(obj, self.typed_catalog)

In [7]:
def initialize_objects(feeder_id, mrid_list):
    
    query_message = """
        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX c:  <http://iec.ch/TC57/CIM100#>
        SELECT ?eqid ?eqname ?class
        {
          VALUES ?fdrid {"%s"}
          VALUES ?eqid {"""%feeder_id
    for mrid in mrid_list:
        query_message += ' "%s" \n'%mrid
    
    query_message += """               } 
          ?fdr c:IdentifiedObject.mRID ?fdrid.
          ?eq c:IdentifiedObject.name ?eqname.
          ?eq c:IdentifiedObject.mRID ?eqid.
          ?eq a ?classraw.
          bind(strafter(str(?classraw),"CIM100#") as ?class)
        }
        GROUP BY  ?eqid ?eqname ?class
        ORDER by  ?fdrid
        """
    #print(query_message)
    results = gapps.query_data(query = query_message, timeout = 60)
    output = results['data']['results']['bindings']

    #if self.sparql is None:
    #    self.sparql = SPARQLWrapper(self.url)
    #    self.sparql.setReturnFormat(JSON)
    #self.sparql.setQuery(sparql)

    #ret = self.sparql.query().convert()
    objects = []
    for result in output:
        cls = result['class']['value']
        mrid = result['eqid']['value']
        try:
            objects.append(eval(f"{cls}(mRID='{mrid}')"))
            print(cls)
        except:
            print('warning: object class missing:', cls)
    return objects

In [8]:
feeder_obj = Feeder(mRID = feeder_mrid)
feeder_obj

Feeder(mRID='_5B816B93-7A5F-B64C-8460-47C17D6E4B0F', aliasName=None, description=None, name=None, Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=None, Measurements=[], ConnectivityNodes=[], TopologicalNode=[], AdditionalGroupedEquipment=[], Equipments=[], NamingSecondarySubstation=[], NormalEnergizedSubstation=[], NormalEnergizingSubstation=None, NormalHeadTerminal=[])

In [9]:

network = DistributedModel(feeder_obj)


In [10]:
network.get_topology_api_response()

{'feeders': {'feeder_id': '_5B816B93-7A5F-B64C-8460-47C17D6E4B0F',
  'addressable_equipment': ['_4E1B3F09-CB88-4A5E-8198-24490EE7FC58',
   '_28456F60-7196-47E4-9BE6-54F7EAABC04A',
   '_34A04257-8486-40AC-BE74-4022F7D395E4'],
  'unaddressable_equipment': ['_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB',
   '_3a9b6186-7f25-48ec-ae18-7d705ff1fa95',
   '_39905671-1860-435f-876d-171b6d7dc312',
   '_3aed8640-d654-4e8b-8fae-c655f07253ce',
   '_982d3a8d-32c6-4502-8093-c638c50d0fa7',
   '_18b33a13-c766-48f2-af7a-09027fd38693',
   '_4e706e09-bcd3-4177-8936-8981552cfd0f',
   '_6fa6beec-44fd-487c-b8ec-592c5e2e7ff4',
   '_ccd7dc0e-8a0d-4414-a001-8cbdcc9a51dc',
   '_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB',
   '_729c2465-26a7-4b17-8c1d-ed735c40feb1',
   '_a66e189b-e425-4e3c-9b0c-0315eceb6240',
   '_37e4eb75-0ac0-4cc2-baac-2c084d34d713',
   '_5beee6b8-cfb3-46ec-b5c2-434411809e1c',
   '_8b3143b4-f8aa-4d26-baed-a065b22152a2',
   '_e0ef9f1d-d749-4232-b347-c34e34929b11',
   '_3f60c60a-f30b-4a60-ac8c-5f537524b387',


In [11]:
network.initialize_network()

LinearShuntCompensator
PowerTransformer
LoadBreakSwitch
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch


In [12]:
def load_all_attributes(obj_list, obj_type):
    if obj_type == ACLineSegment:
        # load_aclinesegment(obj_list)
        print('getting lines')
    elif obj_type == LinearShuntCompensator:
        load_all_capacitor_attributes(obj_list)

In [31]:
#@dataclass
#class LinearShuntCompensatorLoader:
def load_all_capacitor_attributes(feeder_id, capacitors: dict[str, LinearShuntCompensator] | list[str]):
    # Obtain list of mRID strings
    if type(capacitors) == list:
        mrid_list = capacitors
    elif type(capacitors) == dict:
        mrid_list = list(capacitors.keys())
    else:
        mrid_list = []
        for obj in capacitors: mrid_list.append(obj.mRID)
    # Query for all attributes    
    # query prefix
    query_message = """
        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT ?mRID ?name ?BaseVoltage ?Location ?RegulatingControl ?aVRDelay ?b0PerSection 
        ?bPerSection ?controlEnabled ?g0PerSection ?gPerSection ?grounded ?maximumSections ?nomU 
        ?normalSections ?phaseConnection ?sections ?Terminals ?ShuntCompensatorPhase 
        (group_concat(distinct ?Measurement; separator=";") as ?Measurements) 
        WHERE { ?eq r:type cim:LinearShuntCompensator.
         VALUES ?fdrid {"%s"}
          VALUES ?mRID {"""%feeder_id
    # add all equipment mRID
    for mrid in mrid_list:
        query_message += ' "%s" \n'%mrid
    # add all attributes
    query_message += """               } 
        ?eq cim:Equipment.EquipmentContainer ?fdr. #filter by feeder
        ?fdr cim:IdentifiedObject.mRID ?fdrid. #filter by feeder
        ?eq cim:IdentifiedObject.mRID ?mRID. #get cap mrid
        ?eq cim:IdentifiedObject.name ?name. #get cap name
        
        ?eq cim:ConductingEquipment.BaseVoltage ?bv. #get basevoltage object
        ?bv cim:BaseVoltage.nominalVoltage ?BaseVoltage. #get basevoltage value
        
        OPTIONAL {?eq cim:PowerSystemResource.Location ?loc. #get location object
            ?loc cim:IdentifiedObject.mRID ?Location.} #get location mrid
            
        OPTIONAL {?ctl cim:RegulatingControl.RegulatingCondEq ?eq. #get control
            ?ctl cim:RegulatingControl.enabled ?controlEnabled. #get enabled
            ?ctl cim:IdentifiedObject.mRID ?RegulatingControl} #get control mrid
        
        ?t cim:Terminal.ConductingEquipment ?eq.
        ?t cim:IdentifiedObject.mRID ?Terminals
        OPTIONAL {?meas cim:Measurement.Terminal ?t.
            ?meas cim:IdentifiedObject.mRID ?Measurement}

        OPTIONAL {?eq cim:ShuntCompensator.grounded ?grounded.}
        OPTIONAL {?eq cim:ShuntCompensator.maximumSections ?maximumSections.}
        OPTIONAL {?eq cim:ShuntCompensator.nomU ?nomU.}
        OPTIONAL {?eq cim:ShuntCompensator.normalSections ?normalSections.}
        OPTIONAL {?eq cim:ShuntCompensator.aVRDelay ?aVRDelay.}
        OPTIONAL {?eq cim:ShuntCompensator.phaseConnection ?phaseConn.
            bind(strafter(str(?phaseConn),"CIM100#") as ?phaseConnection)}
        OPTIONAL {?spc cim:ShuntCompensatorPhase.ShuntCompensator ?eq.
            ?spc cim:IdentifiedObject.mRID ?ShuntCompensatorPhase.}

        OPTIONAL {?eq cim:LinearShuntCompensator.b0PerSection ?b0PerSection.}
        OPTIONAL {?eq cim:LinearShuntCompensator.bPerSection ?bPerSection.}
        OPTIONAL {?eq cim:LinearShuntCompensator.g0PerSection ?g0PerSection.}
        OPTIONAL {?eq cim:LinearShuntCompensator.gPerSection ?gPerSection.}
        }
        GROUP by  ?mRID ?name ?BaseVoltage ?Location ?RegulatingControl ?aVRDelay ?b0PerSection 
        ?bPerSection ?controlEnabled ?g0PerSection ?gPerSection ?grounded ?maximumSections ?nomU
        ?normalSections ?phaseConnection ?sections ?Terminals ?ShuntCompensatorPhase
        ORDER by  ?name
    """
    #print(query_message)
    results = gapps.query_data(query = query_message, timeout = 60)
    output = results['data']['results']['bindings']
    
    #print(output)

    for index in range(len(output)):
        mRID = output[index]['mRID']['value']
        name = output[index]['name']['value']
        query_string_parser(capacitors, mRID, output[index], 'BaseVoltage')
        query_string_parser(capacitors, mRID, output[index], 'Location')
        query_string_parser(capacitors, mRID, output[index], 'aVRDelay')
        query_string_parser(capacitors, mRID, output[index], 'aliasName')
        query_string_parser(capacitors, mRID, output[index], 'b0PerSection')
        query_string_parser(capacitors, mRID, output[index], 'bPerSection')
        query_string_parser(capacitors, mRID, output[index], 'controlEnabled')
        query_string_parser(capacitors, mRID, output[index], 'description')
        query_string_parser(capacitors, mRID, output[index], 'g0PerSection')
        query_string_parser(capacitors, mRID, output[index], 'gPerSection')
        query_string_parser(capacitors, mRID, output[index], 'grounded')
        query_string_parser(capacitors, mRID, output[index], 'inService')
        query_string_parser(capacitors, mRID, output[index], 'mRID')
        query_string_parser(capacitors, mRID, output[index], 'maximumSections')
        query_string_parser(capacitors, mRID, output[index], 'name')
        query_string_parser(capacitors, mRID, output[index], 'nomU')
        query_string_parser(capacitors, mRID, output[index], 'normalSections')
        query_string_parser(capacitors, mRID, output[index], 'phaseConnection')
        query_string_parser(capacitors, mRID, output[index], 'sections')
        
        query_list_parser(capacitors, mRID, output[index], 'Measurements', ';')
        query_list_parser(capacitors, mRID, output[index], 'Terminals', ';')
        
        query_class_parser(capacitors, mRID, output[index], 'RegulatingControl')
        query_class_parser(capacitors, mRID, output[index], 'ShuntCompensatorPhase')

In [14]:
# def query_string_parser(obj_dict, mRID, query, key):
#     try:
#         setattr(obj_dict[mRID], key, query[key]['value'])
#     except:
#         []

In [24]:
# def query_class_parser(obj_dict, mRID, query, class_name):
#     try: 
#         cls = class_name
#         setattr(obj_dict[mRID], class_name, eval(f"{cls}(mRID='{query[class_name]['value']}')"))
#     except:
#         []

In [16]:
# def query_list_parser(obj_dict, mRID, query, key, separator):
#     #print(query[key]['value'].split(separator))
#     try: 
#         setattr(obj_dict[mRID], key, query[key]['value'].split(separator))
#     except:
#         []

In [18]:
d = network.switch_areas[0].typed_catalog[LinearShuntCompensator]
d

{'_6A7D7AF8-A0F7-41B6-BB0C-160EAE054DEA': LinearShuntCompensator(mRID='_6A7D7AF8-A0F7-41B6-BB0C-160EAE054DEA', aliasName=None, description=None, name=None, Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=None, Measurements=[], inService=None, Faults=[], OperationalLimitSet=[], BaseVoltage=None, SvStatus=[], Terminals=[], controlEnabled=None, RegulatingControl=None, aVRDelay=None, grounded=None, maximumSections=None, nomU=None, normalSections=None, phaseConnection=None, sections=None, ShuntCompensatorPhase=[], SvShuntCompensatorSections=None, b0PerSection=None, bPerSection=None, g0PerSection=None, gPerSection=None)}

In [32]:
load_all_capacitor_attributes(feeder_mrid, d)

In [40]:
network.switch_areas[0].unaddressable_equipment

{'_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB': ACLineSegment(mRID='_6EBB2188-CBE1-4FEC-9406-83DE8E61F9CB', aliasName=None, description=None, name=None, Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=None, Measurements=[], inService=None, Faults=[], OperationalLimitSet=[], BaseVoltage=None, SvStatus=[], Terminals=[], length=None, b0ch=None, bch=None, g0ch=None, gch=None, r=None, r0=None, x=None, x0=None, ACLineSegmentPhases=[], PerLengthImpedance=None, WireSpacingInfo=None)}